# CS 506 HW1 Solution
Name:

### 1. Understandking K-means Clustering

(Please fill out the functions in k_means_clustering.py)

### 2. Working with the Algorithms

In [3]:
from typing import List, Dict, Tuple
# feel free to add more functions such as discard missing examples

import pandas as pd
import numpy as np

def read_dataset(dataset_path: str):
    """
    read in NYC dataset and return a dataset type of your choice
    :param dataset_path: the string path to the dataset file
    :return: dataset
    """ 
    df = pd.read_csv(dataset_path)
    df.dropna(subset = ['latitude','longitude','price'])
    df = df[['latitude','longitude','price']]
    return df

from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
import sklearn.metrics as metrics

from sklearn.cluster import AgglomerativeClustering
import scipy.cluster
import scipy.cluster.hierarchy as hierarchy
from scipy.cluster.hierarchy import centroid, fcluster
import scipy.spatial.distance
from sklearn.neighbors import NearestCentroid
import sklearn.mixture
from sklearn.preprocessing import MinMaxScaler

def scale_dataset():
    dataset = read_dataset('listings.csv')
    scaler = MinMaxScaler()
    dataset[dataset.columns] = scaler.fit_transform(dataset[dataset.columns])
    return dataset

def cluster_kmeans():
    fix_dataset = scale_dataset()
    fix_dataset = fix_dataset.iloc[:,:].values   
    # determine the number of K, by ploting the graph, error does not decrease
    # fast after K = 11, so setting K as 11.
    
    error = np.zeros(15)
    for k in range(1,15):
        kmeans = KMeans(init='k-means++', n_clusters = k, n_init = 100)
        kmeans.fit_predict(fix_dataset)
        error[k] = kmeans.inertia_
    
    plt.plot(range(1, len(error)), error[1:], 'o-')
    plt.xlabel('Number of clusters')
    plt.title(r'$k$-means clustering performance')
    plt.ylabel('Error');
    
    kmeans = KMeans(init = 'k-means++', n_clusters = 11, n_init = 100)
    k_means = kmeans.fit_predict(fix_dataset)
    return k_means

def cluster_hierar():        
    fix_dataset = scale_dataset()
    fix_dataset = fix_dataset.iloc[:,:].values
    
    # choose cluster as 5 because there are five neighborhoods in the area.
    error = np.zeros(7)
    for k in range(1,7):
        hierar = AgglomerativeClustering(n_clusters = 5, method = 'average')
        hierar.fit_predict(fix_dataset)
        error[k] = hierar.inertia_
    
    plt.plot(range(1, len(error)), error[1:], 'o-')
    plt.xlabel('Number of clusters')
    plt.title(r'$k$-means clustering performance')
    plt.ylabel('Error');
    
    
    # Using ward because it is less susceptible to noise and outliers, which probably contains in the dataset
    # because some points are far to the the major group.
    hierar = AgglomerativeClustering(n_clusters = 5, affinity = 'euclidean', linkage ='ward')
    hierar_labels = hierar.fit_predict(fix_dataset)
    return hierar_labels

def cluster_GMM():
    fix_dataset = scale_dataset()
    fix_dataset = fix_dataset.iloc[:,:].values  
    
    # choose cluster as 5, as the first high bump of Silhouette Score graph is at 5. Covariance_type is full 
    # because each state has its own general covariance matrix.
    
    max_clusters = 10
    s = np.zeros(max_clusters+1)
    for k in range(2, max_clusters+1):
        gmm = sklearn.mixture.GaussianMixture(n_components=5, covariance_type='full')
        gmm_labels = gmm.fit_predict(fix_dataset)
        s[k] = metrics.silhouette_score(fix_dataset, gmm_labels, metric = 'euclidean')
    plt.plot(range(2, len(s)), s[2:])
    plt.xlabel('Number of clusters')
    plt.ylabel('Silhouette Score');
    return s
    
    gmm = sklearn.mixture.GaussianMixture(n_components=5, covariance_type='full')
    gmm_labels = gmm.fit_predict(fix_dataset)
    return gmm_labels

def cluster_nyc_listings():
    """
    cluster AirBnb listings using k-means++, hierarchical, and GMM
    :return: 
    """
    return cluster_kmeans(), cluster_hierar(), cluster_GMM()  

### 2b List a few bullet points describing the pros and cons of the various clustering algorithms.

Kmeans:
Ad: It guarantees convergence
It scales to large data sets.

Disad: It tries to find spherical clusters
It tries to find equal_sized clusters
Good at elliptical graphs
It is sensitive to the starting cluster centers

Hierarchical:
Ad: Easy to decide the number of clusters on the dendrogram
Single-linkage clustering can handle non_elliptical shapes

Disad: Not suitable for large dataset because of time complexity
Initial seeds can be influential to the result
Sensitive to outliers

GMM:
Ad: With GMM, each cluster can have unconstrained covariance structure, so cluster assignment is much more flexible in GMM than in k-means, like elongated or skewed groups can be assigned together.
GMM allows for mixed membership of points to clusters. A data does not necessarily belongs to one cluster in the beginning.

Disad:
It takes a lot of time to run
The scaling can be biased sometimes

### 3 Data Visualization
### 3a Produce a Heatmap. Is this heatmap useful in order to draw conclusions about the expensiveness of areas within NYC? if not, why?

In [ ]:
import folium
import pandas as pd
from pandas import Series, DataFrame
import seaborn as sns
from folium.plugins import HeatMap

df = pd.read_csv('listings.csv')

def generate_base_map(default_location = [40.693943, -73.985880]):
    base_map = folium.Map(location=default_location)
    return base_map
                      
base_map = generate_base_map()
HeatMap(
    data=df[["latitude", "longitude", "price"]]
    .groupby(["latitude", "longitude"])
    .mean()
    .reset_index()
    .values.tolist(),
    radius=8,
    max_zoom=13,
).add_to(base_map)
base_map.save("index.html")

Yes but not strong, because the dataset are group by its latitude and longitude, so the price level is somewhat reflective on the 2-D map. Although the color does not vary that much, there is still a clear concentrated area in middle part of Manhattan, which is true in real situation.

### 3b Visualize the clusters by plotting the longitude / lattitude of every listings in a scatter plot

In [ ]:
import seaborn as sns

def visualize_kmeans_clusters():
    df = read_dataset('listings.csv')
    df_xy = df[['longitude','latitude']]
    clusters = cluster_kmeans()
    longitude = df[['longitude']].to_numpy().reshape(-1)
    latitude = df[['latitude']].to_numpy().reshape(-1)
    df_final = pd.DataFrame({'longitude': longitude, 'latitude': latitude, 'clusters': clusters})
    plot = sns.scatterplot(x = 'longitude', y = 'latitude', data = df_xy, size = 0.0005, hue = clusters)
    return plot

def visualize_hierar_clusters():
    df = read_dataset('listings.csv')
    df_xy = df[['longitude','latitude']]
    clusters = cluster_hierar()
    longitude = df[['longitude']].to_numpy().reshape(-1)
    latitude = df[['latitude']].to_numpy().reshape(-1)
    df_final = pd.DataFrame({'longitude': longitude, 'latitude': latitude, 'clusters': clusters})
    plot = sns.scatterplot(x = 'longitude', y = 'latitude', data = df_xy, size = 0.0005, hue = clusters)
    return plot

def visualize_GMM_clusters():
    df = read_dataset('listings.csv')
    df_xy = df[['longitude','latitude']]
    clusters = cluster_GMM()
    longitude = df[['longitude']].to_numpy().reshape(-1)
    latitude = df[['latitude']].to_numpy().reshape(-1)
    df_final = pd.DataFrame({'longitude': longitude, 'latitude': latitude, 'clusters': clusters})
    plot = sns.scatterplot(x = 'longitude', y = 'latitude', data = df_xy, size = 0.0005, hue = clusters)
    return plot

def visualize_clusters():
    visualize_kmeans_clusters(), visualize_hierar_clusters(), visualize_GMM_clusters()
    return

### 3c For every cluster, report the average price of the listings within this cluster

In [ ]:
def average_price_kmeans():
    dataset = read_dataset('listings.csv')
    fix_dataset = dataset.iloc[:,:].values
    cluster = cluster_kmeans()

    total = [0] * 11
    num = [0] * 11
    avg = [0] * 11
    label = 0
    col_price_list = dataset['price'].tolist()
    for index in range(len(cluster)):
        label = cluster[index]
        total[label] = total[label] + col_price_list[index]
        num[label] = num[label] + 1
    for i in range(len(total)):
        avg[i] = total[i] / num[i]
    return avg

def average_price_hierar():
    dataset = read_dataset('listings.csv')
    fix_dataset = dataset.iloc[:,:].values
    cluster = cluster_hierar()

    total = [0] * 5
    num = [0] * 5
    avg = [0] * 5
    label = 0
    col_price_list = dataset['price'].tolist()
    for index in range(len(cluster)):
        label = cluster[index]
        total[label] = total[label] + col_price_list[index]
        num[label] = num[label] + 1
    for i in range(len(total)):
        avg[i] = total[i] / num[i]
    return avg

def average_price_GMM():
    dataset = read_dataset('listings.csv')
    fix_dataset = dataset.iloc[:,:].values
    cluster = cluster_GMM()

    total = [0] * 5
    num = [0] * 5
    avg = [0] * 5
    label = 0
    col_price_list = dataset['price'].tolist()
    for index in range(len(cluster)):
        label = cluster[index]
        total[label] = total[label] + col_price_list[index]
        num[label] = num[label] + 1
    for i in range(len(total)):
        avg[i] = total[i] / num[i]
    return avg

### 3d Bonus point (provide a plot on an actual NYC map)

### 3e Are the findings in agreement with what you have in mind about the cost of living for neighborhoods in NYC? If you are unfamiliar with NYC, you can consult the web.

In [ ]:
Yes. In my map, I found the highest average cost orrurs at Mid-part and down-part of Manhattan, then Brooklyn. ronx has a lower
price, which fits the situation.

### 4. Image Manipulation

In [ ]:
# You need to add more functions here

import cv2
import numpy as np
from sklearn.cluster import KMeans
import k_means_clustering as kmc

import random
import matplotlib.pyplot as plt
import scipy.io
import scipy.misc
import pdb

import cv2
import numpy as np
from sklearn.cluster import KMeans
import k_means_clustering as kmc

import random
import matplotlib.pyplot as plt
import scipy.io
import scipy.misc
import pdb

def cluster_k_means(image_path:str):
    img = cv2.imread(image_path)
    img = img.reshape((850 * 1280, 3))    
    clusters, centroid_history = kmc.run_k_means(img, kmc.choose_random_centroids(img, K=10), n_iter=10)
    new_img = img.copy()
    for i in range(len(clusters)):
        new_img[i] = centroid_history[-1][clusters[i]]

    final_img = np.array(new_img)
    img = final_img.reshape((850, 1280, 3))
    return img

def show_image(image):
    cv2.imshow('Display Window', image) 
    cv2.waitKey(0)
    cv2.destroyAllWindows()